In [5]:
import pandas as pd
import numpy as np
from mca import diagsvd
import functools
import sklearn.model_selection as ms
from sklearn.preprocessing import StandardScaler as stdc
import gc
import _pickle as pickle
categorical = ['source_system_tab','source_type','gender','city']
numerical = ['song_length', 'song_year']

def standarize_data(df):
	df[numerical] = stdc().fit_transform(df[numerical])
	return df



In [11]:
def split(X, y, proportion):
	print('Train \nRaw data: \n ', 'Shape: ', X.shape, 'Type: ', type(X))
	print('Target \nRaw data: \n ', 'Shape: ', y.shape)
	# I split the data between the X and the target value

	# X is a dataframe not a np.array
	print('Size of X: ', X.shape, '\nSize of y: ', y.shape)
	(X_train, X_test, y_train, y_test) = ms.train_test_split(X, y, test_size=proportion, random_state=1, stratify=y)
	(X_test, X_val, y_test, y_val) = ms.train_test_split(X_test, y_test, test_size=.5, random_state=1, stratify=y_test)
	print('\n New train shape: ', X_train.shape, ' \n New test shape: ', X_test.shape, '\n New val shape: ',
	      X_val.shape, '\n Type: ',type(X_train))

	return X_train, X_test, X_val, y_train, y_test, y_val


# Partitioning

In [10]:
file = 'def_training.csv'
X = pd.read_csv('../Data/' + file)
X['city'] = X['city'].astype('category')
y = X['target']

X_num = X[numerical]
X_num = standarize_data(X_num)

T = pd.read_csv('../Data/def_test.csv')
T['city'] = T['city'].astype('category')

T_num = T[numerical]
T_num = standarize_data(T_num)

DummiesX = pd.get_dummies(data=X[categorical], prefix_sep='|')
T_cat = pd.get_dummies(data=T[categorical], prefix_sep='|')

print(T_cat.shape)
print(DummiesX.shape)

# generate_partition(X)
# preprocess()

/home/raquel/.local/lib/python3.5/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
/home/raquel/.local/lib/python3.5/site-packages/pandas/core/frame.py:2544: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.loc._setitem_with_indexer((slice(None), indexer), value)
/home/raquel/.local/lib/python3.5/site-packages/pandas/core/frame.py:2514: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-

   gender  source_system_tab|discover  source_system_tab|explore  \
0       0                           0                          0   
1       0                           0                          0   
2       1                           1                          0   
3       1                           0                          0   
4       1                           0                          0   

   source_system_tab|listen with  source_system_tab|my library  \
0                              0                             1   
1                              0                             1   
2                              0                             0   
3                              0                             0   
4                              0                             0   

   source_system_tab|notification  source_system_tab|radio  \
0                               0                        0   
1                               0                        0   
2       

In [ ]:
(X_num_train, X_num_test, X_num_val, y_train, y_test, y_val) = split(X_num, y, 0.5)


In [39]:
X_cat_val = pd.read_csv('../Data/subset_cat_Val.csv', compact_ints=True)

X_num_val = pd.read_csv('../Data/subset_num_Val.csv')

concat_Val = pd.concat([X_cat_val, X_num_val], axis=1, ignore_index=True)

y_val = pd.read_csv('../Data/preprocessed_y_Val.csv')

fin = pd.concat([concat_Val, y_val], axis=1, ignore_index=True)


/home/raquel/.local/lib/python3.5/site-packages/pandas/io/parsers.py:809: FutureWarning: The 'compact_ints' argument has been deprecated and will be removed in a future version.


  self.options, self.engine = self._clean_options(options, engine)


,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
0,0,1637355,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1637355,0.777325,-1.323276,187559.0,1.0
1,1,187559,0,0,0,0,1,0,0,0,...,0,0,0,0,0,187559,-0.503867,-0.237630,4898129.0,0.0
2,2,4898129,1,1,0,0,0,0,0,0,...,0,0,0,0,0,4898129,-0.721186,0.692924,6544335.0,1.0
3,3,6544335,0,0,0,0,1,0,0,0,...,0,0,0,0,0,6544335,0.911917,-0.702907,6905060.0,0.0
4,4,6905060,1,1,0,0,0,0,0,0,...,0,0,0,0,0,6905060,0.232897,0.848017,1908985.0,0.0


In [45]:
fin = fin.rename(columns ={48:'target'})
fin.columns

Index([       0,        1,        2,        3,        4,        5,        6,
              7,        8,        9,       10,       11,       12,       13,
             14,       15,       16,       17,       18,       19,       20,
             21,       22,       23,       24,       25,       26,       27,
             28,       29,       30,       31,       32,       33,       34,
             35,       36,       37,       38,       39,       40,       41,
             42,       43,       44,       45,       46,       47, 'target'],
      dtype='object')

In [54]:
X_train = pd.read_csv('../Data/preprocessedTrainMCA.csv', compact_ints=True)
X_test = pd.read_csv('../Data/preprocessedTestMCA.csv', compact_ints=True)
X_val = pd.read_csv('../Data/preprocessedValMCA.csv', compact_ints=True)

/home/raquel/.local/lib/python3.5/site-packages/pandas/io/parsers.py:809: FutureWarning: The 'compact_ints' argument has been deprecated and will be removed in a future version.


  self.options, self.engine = self._clean_options(options, engine)


In [55]:
X_train = X_train.rename(columns={'9': 'target'})
X_test = X_test.rename(columns={'9': 'target'})
X_val = X_val.rename(columns={'9': 'target'})

In [3]:
def data_from_filesMCA():
	X_train = pd.read_csv('../Data/preprocessedTrainMCA.csv', compact_ints=True)
	X_test = pd.read_csv('../Data/preprocessedTestMCA.csv', compact_ints=True)
	X_val = pd.read_csv('../Data/preprocessedValMCA.csv', compact_ints=True)

	X_train = X_train.rename(columns={'9': 'target'})
	X_test = X_test.rename(columns={'9': 'target'})
	X_val = X_val.rename(columns={'9': 'target'})

	y_train = X_train['target']
	y_test = X_test['target']
	y_val = X_val['target']
	print('\nLoaded data:')
	X_train = X_train.drop(columns=['target','Unnamed: 0'])
	X_test = X_test.drop(columns=['target','Unnamed: 0'])
	X_val = X_val.drop(columns=['target','Unnamed: 0'])
	print('Train shape: ', X_train.shape)
	print('Train shape Y: ', y_train.shape)
	print('Test shape: ', X_test.shape)
	print('Test shape Y: ', y_test.shape)
	print('Val shape: ', X_val.shape)
	print('Test shape: ', y_val.shape)
	return X_train, X_test, X_val, y_train, y_test, y_val

In [6]:
# X_train = X_train.drop(columns=['target','Unnamed: 0'])
(X_train, X_test, X_val, y_train, y_test, y_val) = data_from_filesMCA()

/home/raquel/.local/lib/python3.5/site-packages/pandas/io/parsers.py:809: FutureWarning: The 'compact_ints' argument has been deprecated and will be removed in a future version.


  self.options, self.engine = self._clean_options(options, engine)



Loaded data:
Train shape:  (3673816, 14)
Train shape Y:  (3673816,)
Test shape:  (1836908, 14)
Test shape Y:  (1836908,)
Val shape:  (1836909, 14)
Test shape:  (1836909,)


In [7]:
X_val

,0,1,2,3,4,5,6,7,8,10,11,12,13,14
0,-8.523199e+03,-8.278610,-7.060705,217.375468,-192.586061,378.386951,-1486.900029,7.508320,-442.170898,1637355,0.777325,-1.323276,187559.0,1.0
1,-9.696597e+02,2.131678,-37.576361,-310.771373,-122.928212,-173.573304,25.001617,-97.757650,-86.947208,187559,-0.503867,-0.237630,4898129.0,0.0
2,-2.549857e+04,5.079773,-1.892851,101.564431,1.336558,25.882835,185.124821,75.774877,-116.487695,4898129,-0.721186,0.692924,6544335.0,1.0
3,-3.406750e+04,11.651822,24.008966,-398.922659,219.484450,-14.991948,118.707995,-65.206299,-215.341821,6544335,0.911917,-0.702907,6905060.0,0.0
4,-3.594565e+04,5.819567,52.147701,69.695288,40.651520,34.163670,186.393325,62.287481,-65.629486,6905060,0.232897,0.848017,1908985.0,0.0
5,-9.927714e+03,-0.067722,-9.991478,15.372140,-47.033958,71.450692,16.101821,-14.399654,5.241432,1908985,-0.488176,0.692924,2399605.0,0.0
6,-1.247787e+04,50.475095,13.194496,-446.014784,343.578777,164.192636,23.358158,-30.006473,12.047711,2399605,1.501575,0.692924,7126900.0,0.0
7,-3.709423e+04,19.862292,-14.189481,-383.679647,-105.768153,-219.924120,-1.283337,-150.955301,-81.157049,7126900,-1.220279,-1.168184,6299556.0,0.0
8,-3.278483e+04,4.261288,0.493681,63.497612,-9.543232,9.379175,114.313958,45.692804,-241.468533,6299556,0.574137,-0.857999,5883283.0,0.0
9,-3.061520e+04,3.195703,-0.909242,67.890481,-10.573104,12.160138,115.887170,48.884173,-241.810357,5883283,-0.989347,0.692924,5616372.0,0.0


In [ ]:
def Matrix_mult(*args):
	"""An internal method to multiply matrices."""
	return functools.reduce(np.dot, args)

# MCA

In [ ]:
def my_mca(X_train):
	size = [20000, X_train.shape[1]]
	N_all = np.sum(X_train.values[0:size[0],:])

	Z_residual = X_train.values[0:size[0],:] / N_all
	# print('zshape', Z.shape)
	Sum_r = np.sum(Z_residual, axis=1)
	Sum_c = np.sum(Z_residual, axis=0)
	# print('sumc', Sum_c)
	print(size, Sum_r.shape, Sum_c.shape, N_all)
	# Compute residual
	Z_expected = np.outer(Sum_r, Sum_c)
	Z_residual -= Z_expected
	del Z_expected
	gc.collect()
	print('Z res')
	# Scale residual by the square root of column and row sums.
	# Note we are computing SVD on residual matrix, not the analogous covariance matrix,
	# Therefore, we are dividing by square root of Sums.
	# D_r = np.diag(Sum_r)
	# D_c = np.diag(Sum_c)
	D_r_sqrt_mi = np.sqrt(np.diag(Sum_r ** -1))
	del Sum_r
	gc.collect()
	D_c_sqrt_mi = np.sqrt(np.diag(Sum_c ** -1))
	del Sum_c
	gc.collect()
	print('end DC Dr')
	# print(Z_residual.shape, Z.shape, D_r_sqrt_mi.shape, D_c_sqrt_mi.shape)
	MCA_mat = Matrix_mult(D_r_sqrt_mi, Z_residual, D_c_sqrt_mi)
	## Apply SVD.
	## IN np implementation, MCA_mat = P*S*Q, not P*S*Q'
	P, S, Q = np.linalg.svd(MCA_mat)
	print(P.shape, S.shape, Q.shape)
	# Verify if MCA_mat = P*S*Q,
	S_d = diagsvd(S, size[0], size[1])
	# sum_mca = np.sum((Matrix_mult(P, S_d, Q) - MCA_mat) ** 2)
	# print('Difference between SVD and the MCA matrix is %0.2f' % sum_mca)
	# Compute factor space, or row and column eigen space
	F = Matrix_mult(D_r_sqrt_mi, P, S_d)  ## Column Space, contains linear combinations of columns
	G = Matrix_mult(D_c_sqrt_mi, Q.T, S_d.T)  ## Row space, contains linear combinations of rows
	pickle.dump(F, open('F.pkl','wb'))
	pickle.dump(G, open('G.pkl', 'wb'))
	pickle.dump(S, open('S.pkl', 'wb'))
	del  Q, P, MCA_mat, S_d, D_r_sqrt_mi, D_c_sqrt_mi
	gc.collect()
	print(F.shape, G.shape)
	Lam = S ** 2
	Expl_var = Lam / np.sum(Lam)

	print('Eigen values are ', Lam)
	print('Explained variance of eigen vectors are ', Expl_var)
	# K = 10
	# E = np.array([(K / (K - 1.) * (lm - 1. / K)) ** 2 if lm > 1. / K else 0 for lm in S ** 2])
	# Expl_var_bn = E / np.sum(E)
	# print('Eigen vectors after Benzécri correction are ', E)
	# print('Explained variance of eigen vectors after Benzécri correction are ', Expl_var_bn)
	# J = 22.
	# green_norm = (K / (K - 1.) * (np.sum(S ** 4) - (J - K) / K ** 2.))
	# # J is the number of categorical variables. 22 in our case.

	# print('Explained variance of eigen vectors after Greenacre correction are ', E / green_norm)
	data = {'Iλ': pd.Series(Lam),
		'τI': pd.Series(Expl_var),
		# 'Zλ': pd.Series(E),
		# 'τZ': pd.Series(Expl_var_bn),
		# 'cλ': pd.Series(E),
		# 'τc': pd.Series(E / green_norm),
	}
	columns = ['Iλ', 'τI']#, 'Zλ', 'τZ', 'cλ', 'τc']
	table2 = pd.DataFrame(data=data, columns=columns).fillna(0)
	table2.index += 1
	table2.loc['Σ'] = table2.sum()
	table2.index.name = 'Factor'
	np.round(table2.astype(float), 4)
	table2.to_csv('table2.csv')
	columns_keep = 5
	print('Varianza explicada tomando 40 features: ', table2['τI'][0:columns_keep].sum())
	## The projection can also be computed using vectorized form,

	X_train = Matrix_mult(X_train.values, G[:, :columns_keep]) / S[:columns_keep] / 10
	X_test = pd.read_csv('../Data/subset_cat_Test.csv', compact_ints=True)
	X_test = Matrix_mult(X_test.values, G[:, :columns_keep]) / S[:columns_keep] / 10
	X_val = pd.read_csv('../Data/subset_cat_Val.csv', compact_ints=True)
	X_val = Matrix_mult(X_val.values, G[:, :columns_keep]) / S[:columns_keep] / 10
	T_cat = pd.read_csv('../Data/subset_cat_T.csv')
	T_cat = Matrix_mult(T_cat.values, G[:, :columns_keep]) / S[:columns_keep] / 10
	print('shape', X_test.shape)
	names = ['V' + str(i) for i in range(columns_keep)]
	return pd.DataFrame(X_train, columns=names), pd.DataFrame(X_test, columns=names), pd.DataFrame(X_val, columns=names),  pd.DataFrame(T_cat, columns=names)


In [ ]:
def split(X, y, proportion):
	print('Train \nRaw data: \n ', 'Shape: ', X.shape, 'Type: ', type(X))
	print('Target \nRaw data: \n ', 'Shape: ', y.shape)
	# I split the data between the X and the target value

	# X is a dataframe not a np.array
	print('Size of X: ', X.shape, '\nSize of y: ', y.shape)
	(X_train, X_test, y_train, y_test) = ms.train_test_split(X, y, test_size=proportion, random_state=1, stratify=y)
	(X_test, X_val, y_test, y_val) = ms.train_test_split(X_test, y_test, test_size=.5, random_state=1, stratify=y_test)
	print('\n New train shape: ', X_train.shape, ' \n New test shape: ', X_test.shape, '\n New val shape: ',
	      X_val.shape, '\n Type: ',type(X_train))

	return X_train, X_test, X_val, y_train, y_test, y_val


In [ ]:
def generate_partition(X):
	Test = pd.read_csv('../Data/def_test.csv')
	T_num = Test[numerical]

	X_num = X[numerical]
	X_num = standarize_data(X_num)
	X['city'] = X['city'].astype('category')

	DummiesX = pd.get_dummies(data=X[categorical], prefix_sep='|')
	T_cat = pd.get_dummies(data=Test[categorical], prefix_sep='|')
	y = X['target']
	print('dummies size: ', DummiesX.shape)
	(X_num_train, X_num_test, X_num_val, y_train, y_test, y_val) = split(X_num, y, 0.5)
	del X_num, X
	gc.collect()
	X_num_train.to_csv('../Data/subset_num_Train.csv')
	X_num_test.to_csv('../Data/subset_num_Test.csv')
	X_num_val.to_csv('../Data/subset_num_Val.csv')
	T_num.to_csv('../Data/subset_num_T.csv')
	y_train.to_csv('../Data/preprocessed_y_Train.csv')
	y_test.to_csv('../Data/preprocessed_y_Test.csv')
	y_val.to_csv('../Data/preprocessed_y_Val.csv')

	(X_cat_train, X_cat_test, X_cat_val, y_train, y_test, y_val) = split(DummiesX, y, 0.5)
	del DummiesX
	gc.collect()
	print('end split')
	X_cat_train[X_cat_train.columns] = X_cat_train[X_cat_train.columns].astype(np.uint8)
	X_cat_test[X_cat_test.columns] = X_cat_test[X_cat_test.columns].astype(np.uint8)
	X_cat_val[X_cat_val.columns] = X_cat_val[X_cat_val.columns].astype(np.uint8)
	T_cat[T_cat.columns] = T_cat[T_cat.columns].astype(np.uint8)
	T_cat.to_csv('../Data/T_cat.csv')
	X_cat_train.to_csv('../Data/X_cat_Train.csv')
	X_cat_test.to_csv('../Data/X_cat_Test.csv')
	X_cat_val.to_csv('../Data/X_cat_Val.csv')
	print('loading X train... ')
	X_cat_train = pd.read_csv('../Data/X_cat_Train.csv', compact_ints=True)
	print('loaded')

	T_cat = pd.read_csv('../Data/T_cat.csv', compact_ints=True)


	T_sum =  (T_cat.values.sum(axis=0) != 0)
	X_train_sum = (X_cat_train.values.sum(axis=0) != 0)
	X_cat_test = pd.read_csv('../Data/X_cat_Test.csv', compact_ints=True)
	print('loaded')
	X_test_sum = (X_cat_test.values.sum(axis=0) != 0)
	print('x_test end')
	print('loading X val... ')
	X_cat_val = pd.read_csv('../Data/X_cat_Val.csv', compact_ints=True)
	print('loaded')
	X_val_sum = (X_cat_val.values.sum(axis=0) != 0)
	print('x val end')
	index =np.logical_and(np.logical_and(X_train_sum, np.logical_and(X_test_sum, X_val_sum)),T_sum)
	print('index end')
	X_cat_train = X_cat_train.loc[:, index]
	X_cat_test = X_cat_test.loc[:, index]
	X_cat_val = X_cat_val.loc[:, index]
	T_cat = T_cat.loc[:,index]
	gc.collect()
	print('loc end')
	T_cat.to_csv('../Data/subset_cat_T.csv')
	X_cat_train.to_csv('../Data/subset_cat_Train.csv')
	X_cat_test.to_csv('../Data/subset_cat_Test.csv')
	X_cat_val.to_csv('../Data/subset_cat_Val.csv')
	del X_cat_val,X_cat_train, X_cat_test, X_num_test, X_num_train, X_num_val, T_cat, T_num, T_sum
	gc.collect()